In [0]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.optimizers import Adam
import tensorflow as tf
from keras.optimizers import SGD
import numpy
import cv2 as cv
import os
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

In [0]:
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 3
# Количество эпох для обучения
nb_epoch = 20
# Размер изображений
img_rows, img_cols = 128, 128
# Количество каналов в изображении: RGB
img_channels = 3

In [0]:
def processData(way, iterImg):
  # инициализирую картинки
  img = cv.imread(way + '/' + os.listdir(way)[i]) 
  # изменяю размер изображения
  img = cv.resize(img, (img_rows, img_cols), interpolation=cv.INTER_AREA)
  return img

# количество изображений в одной папке
n = 1600 

# инициализация массивов для представления картинок в численном виде
arrCans = []
arrBottles = []
arrPapers = []

# добавление массивов пикселов в массив картинок
for i in range(n):
  arrCans.append(processData(os.path.join(os.getcwd(), 'cans'), i))
  arrBottles.append(processData(os.path.join(os.getcwd(), 'bottles'), i))
  arrPapers.append(processData(os.path.join(os.getcwd(), 'papers'), i))

In [0]:
X = [] # набор признаков
Y = [] # набор меток

# присоединение массивов изображений разных классов
X = numpy.array(arrCans + arrBottles + arrPapers)

# нехитрое создание индексной зависимости меток от признаков (признаку i-ного элемента соответствует i-ный элемент метки) 
for i in range(nb_classes * n):
	if i <= 1599:
		Y.append(0)
	if i > 1599 and i <= 3199:
		Y.append(1)
	if i > 3199:
		Y.append(2)
# изменение списка в массив numpy
Y = numpy.array(Y)

In [0]:
# инициализация параметров, тренировочных/тестовых признаков и меток
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 42, shuffle = True)

In [0]:
# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [0]:
# Преобразуем метки в категории
Y_train = np_utils.to_categorical(Y_train, nb_classes)
Y_test = np_utils.to_categorical(Y_test, nb_classes)

In [0]:
# инициализация структуры и весов нейронки vgg16
vgg16_net = VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channels))

In [0]:
# запрещаем слою vgg16 обучаться (он уже обучен)
vgg16_net.trainable = False

In [0]:
# вывод структуры vgg16
vgg16_net.summary()

In [0]:
# инициализация модели нейросети
model = Sequential()
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [0]:
# вывод структуры модели
model.summary()

In [0]:
# Задаем параметры оптимизации
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# opt = Adam(lr=0.001)
opt = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [0]:
# Обучаем модель
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)

In [0]:
# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

In [0]:
# вывод того, что нейросеть не определила. Вывод размера тестовой выборки. Дальше вероятность классов, реальный вывод (can, bottle, paper) и правильный вывод
# %matplotlib inline
# import matplotlib.pyplot as plt
# predictions = model.predict(X_test)
# print(predictions.shape)
# iterNum = 0
# for i in range(len(predictions)):
#   maxPred = predictions[i].max()
#   for j in range(len(predictions[i])): 
#     if maxPred == predictions[i][j]: 
#       maxIdxPred = j
#     if Y_test[i].max() == Y_test[i][j]:
#       maxIdxY = j
#   if maxIdxPred != maxIdxY:
#     print(predictions[i])
#     print(numpy.argmax(predictions[i]))
#     print(numpy.argmax(Y_test[i]))
#     plt.imshow(X_test[i])
#     plt.show()
#     iterNum += 1
# print(iterNum)

In [0]:
# сохранение модели нейросети
model.save('trash_cnn.h5')